In [ ]:
import pandas as pd
data=pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv')
data.head()

In [ ]:
data.info()

In [ ]:
data.nunique()

In [ ]:
data.date=pd.to_datetime(data.date, infer_datetime_format=True)
data.info()

In [ ]:
!pip install fast_ml
from fast_ml.feature_engineering import FeatureEngineering_DateTime
dt_fe = FeatureEngineering_DateTime()
dt_fe.fit(data, datetime_variables=['date'])
data = dt_fe.transform(data)
data=data.drop(['date','date:time', 'date:hour', 'date:minute', 'date:second','date:day_part'],axis=1)
data['date:quarter']= 'QQ'+data['date:quarter'].astype('str')
data['date:month'] = 'MM'+data['date:month'].astype('str')
data['date:day'] = 'DD'+data['date:day'].astype('str')
data['date:day_of_week']= 'DW'+data['date:day_of_week'].astype('str')
data['date:weekofyear'] = 'WY'+data['date:weekofyear'].astype('str')
data['date:day_of_year'] = 'DY'+data['date:day_of_year'].astype('str')
data['date:year'] = 'Y'+data['date:year'].astype('str')
data.head()

In [ ]:
!pip install pycaret[full]

In [ ]:
from pycaret.regression import *

In [ ]:
sol= setup(data =data, session_id=42,
             target = 'num_sold',
                  normalize = True, 
                  transformation = True, 
                  remove_multicollinearity = True, 
             transform_target = True,
            train_size = 0.8,
          handle_unknown_categorical = True, unknown_categorical_method = 'most_frequent',
           ignore_features=['row_id'],silent=True)

In [ ]:
model1=tune_model(create_model('catboost'),optimize = 'MAPE')
model1

In [ ]:
model2=tune_model(create_model('xgboost'),optimize = 'MAPE')
model2

In [ ]:
model3=tune_model(create_model('lightgbm'),optimize = 'MAPE')
model3

In [ ]:
model4=tune_model(create_model('rf'),optimize = 'MAPE')
model4

In [ ]:
model5=tune_model(create_model('et'),optimize = 'MAPE')
model5

In [ ]:
model_stacknet_5 = stack_models(estimator_list = [model1,model2,model3,model4,model5])
model_stacknet_5

In [ ]:
predict_model(model_stacknet_5)

In [ ]:
test=pd.read_csv("../input/tabular-playground-series-jan-2022/test.csv")
test.head()

In [ ]:
test.date=pd.to_datetime(test.date, infer_datetime_format=True)
test.info()

In [ ]:
dt_fe.fit(test, datetime_variables=['date'])
test = dt_fe.transform(test)
test=test.drop(['date','date:time', 'date:hour', 'date:minute', 'date:second','date:day_part'],axis=1)
test['date:quarter']= 'QQ'+test['date:quarter'].astype('str')
test['date:month'] = 'MM'+test['date:month'].astype('str')
test['date:day'] = 'DD'+test['date:day'].astype('str')
test['date:day_of_week']= 'DW'+test['date:day_of_week'].astype('str')
test['date:weekofyear'] = 'WY'+test['date:weekofyear'].astype('str')
test['date:day_of_year'] = 'DY'+test['date:day_of_year'].astype('str')
test['date:year'] = 'Y'+test['date:year'].astype('str')
test.head()

In [ ]:
predictions=predict_model(model_stacknet_5,data =test)
predictions.head()

In [ ]:
predictions=predictions[['row_id','Label']]
predictions.head()

In [ ]:
sol=pd.read_csv('../input/tabular-playground-series-jan-2022/sample_submission.csv')
sol.head()

In [ ]:
sol.num_sold=predictions.Label
sol.head()

In [ ]:
sol.to_csv('./submission.csv',index=False)

In [ ]:
save_model(model_stacknet_5, './model_stacknet_5')